# Call me back

... aka *keras callbacks* ...

Keras provides a powerfull mechanism for implementing callbacks, i.e., a function that must be called at a specific point of the training process. We can write our own callbacks (in an another meetup), or use some of the predefined keras callbacks.

Get some data and define a model!

In [2]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPool2D, Conv2D, Flatten
from keras.optimizers import Adam


# Prepare the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((-1, 28, 28, 1)) / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)) / 255.0
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

model = Sequential()
model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Let's define some callbacks:

In [3]:
!mkdir -p models
!mkdir -p logs

In [5]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Early Stopping automatically stops the training when a quantity has stopped improving
# If you use the test set as the validation set, you are cheating!!!
stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='min')

# Save the model every 2 epochs!
checkpoint = keras.callbacks.ModelCheckpoint('models/weights.{epoch:02d}-{val_loss:.2f}.hdf5,', monitor='val_loss', verbose=1, save_best_only=False, 
                                             save_weights_only=False, period=2)

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, batch_size=32, 
                                          write_graph=True, write_grads=True, write_images=False)


In [6]:
results = model.fit(x_train, y_train, batch_size=256, epochs=30,
          verbose=2, validation_data=(x_test, y_test), callbacks=[stopper, checkpoint, tensorboard])


Train on 60000 samples, validate on 10000 samples
Epoch 1/30
 - 5s - loss: 0.2005 - acc: 0.9414 - val_loss: 0.0591 - val_acc: 0.9796
Epoch 2/30
 - 4s - loss: 0.0443 - acc: 0.9863 - val_loss: 0.0354 - val_acc: 0.9887

Epoch 00002: saving model to models/weights.02-0.04.hdf5,
Epoch 3/30
 - 4s - loss: 0.0307 - acc: 0.9906 - val_loss: 0.0266 - val_acc: 0.9915
Epoch 4/30
 - 4s - loss: 0.0224 - acc: 0.9930 - val_loss: 0.0242 - val_acc: 0.9927

Epoch 00004: saving model to models/weights.04-0.02.hdf5,
Epoch 5/30
 - 4s - loss: 0.0177 - acc: 0.9944 - val_loss: 0.0264 - val_acc: 0.9911
Epoch 6/30
 - 4s - loss: 0.0143 - acc: 0.9953 - val_loss: 0.0250 - val_acc: 0.9922

Epoch 00006: saving model to models/weights.06-0.02.hdf5,
Epoch 00006: early stopping


In [10]:
!ls models

weights.02-0.03.hdf5,  weights.04-0.02.hdf5,
weights.02-0.04.hdf5,  weights.06-0.02.hdf5,


Start the tensorboard!



In [7]:
!tensorboard --logdir logs

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.5.1 at http://uranus:6006 (Press CTRL+C to quit)
^C


![alt text](tensor1.png "Title")
![alt text](tensor2.png "Title")